# 21

In [198]:
num_keypad = (('7','8','9'),
              ('4','5','6'),
              ('1','2','3'),
              ('X','0','A'))

num_position = (3,2) # row, col 

dir_keypad = (('X', '^', 'A'),
              ('<', 'v', '>'))

dir_position = (0,2) # row, col

In [344]:
from functools import cache

def get_sequences(code, keypad, position):
    # Initialize the current position to the starting position
    current = position
   
    # Known paths: 
    result = ['']

    # Go character by character in the code
    for char in code: 
        # Find the position of the character on the keypad
        char_position = None

        for row in range(len(keypad)):
            for col in range(len(keypad[0])):
                if keypad[row][col] == char:
                    char_position = (row, col)

        # Find all paths from the current position to the character 
        paths = [('', current)] # (path, position)
        path_stubs = []

        while len(paths) > 0: 
            new_paths = []
            for path, position in paths:
                if position[0] < char_position[0]:
                    if keypad[position[0]+1][position[1]] != 'X':
                        new_paths.append((path+'v', (position[0]+1, position[1])))
                elif position[0] > char_position[0]:
                    if keypad[position[0]-1][position[1]] != 'X':
                        new_paths.append((path+'^', (position[0]-1, position[1])))

                if position[1] < char_position[1]:
                    if keypad[position[0]][position[1]+1] != 'X':
                        new_paths.append((path+'>', (position[0], position[1]+1)))
                elif position[1] > char_position[1]:
                    if keypad[position[0]][position[1]-1] != 'X':
                        new_paths.append((path+'<', (position[0], position[1]-1)))

                if position == char_position:
                    path = path + 'A'
                    path_stubs.append(path)
            paths = new_paths

        # print(f"char: {char}", path_stubs)
        # Update the current position 
        current = char_position

        # Add the shortest path to the result
        updated_result = []
        for path in result: 
            for stub in path_stubs:
                
                updated_result.append(path + stub)

        result = updated_result

    return result      


def roots(code):
    return get_sequences(code, num_keypad, num_position)


def second_sequence(code):
    return get_sequences(code, dir_keypad, dir_position)

### Solution

In [329]:
codes = []
with open("input.txt") as f: 
    for line in f: 
        codes.append(line.strip())
codes

['593A', '508A', '386A', '459A', '246A']

In [292]:
result = 0 

for code in codes: 
    second_sequences = []

    for sequence in roots(code):
        #print(sequence)
        second_sequences.extend(second_sequence(sequence))

    second_sequences.sort(key=lambda x: len(x))
    
    third_sequences = []
    for sequence in second_sequences:
        #print(sequence)
        third_sequences.extend(second_sequence(sequence))

    third_sequences.sort(key=lambda x: len(x))

    # compute the complexity 
    complexity = len(third_sequences[0]) * int(code[:-1])

    result += complexity

result

157892

## Part 2

This took some time. 

First, I noticed that after two steps on the dir keyboard, "v<<A"

In [293]:
print(f"Dir keypad: {dir_keypad}")
print(f"Num keypad: {num_keypad}")

Dir keypad: (('X', '^', 'A'), ('<', 'v', '>'))
Num keypad: (('7', '8', '9'), ('4', '5', '6'), ('1', '2', '3'), ('X', '0', 'A'))


In [590]:
# All possible sequences: 
keypad_sequences = {
        'A^': ['<A'], # starts at A, goes left and commits
        'A>': ['vA'], # starts at A, goes down and commits
        'Av': ['<vA', 'v<A'], # starts at A, goes left and down, commits
        'A<': ['v<<A','<v<A'], # starts at A, goes down and left, commits
        '^A': ['>A'], # starts at A, goes right and commits
        '>A': ['^A'], # starts at A, goes up and commits
        'vA': ['>^A', '^>A'], # starts at A, goes right and up, commits
        '<A': ['>>^A', '>^>A'], # starts at A, goes up and right, commits
        'AA': ['A'], # starts at A, commits
        # ---------------^ 
        '^>': ['v>A', '>vA'],
        '^v': ['vA'],
        '^<': ['v<A'],
        '>^': ['<^A', '^<A'],
        'v^': ['^A'],
        '<^': ['>^A'],
        '^^': ['A'],
        # ---------------v 
        'v<': ['<A'],
        '<v': ['>A'],
        'v>': ['>A'],
        '>v': ['<A'],
        'vv': ['A'],
        # ---------------<
        '<>': ['>>A'],
        '><': ['<<A'],
        '<<': ['A'],
        # --------------->
        '>>': ['A'],
    }   

## Precompute optimal sequences on the dir keypad

In [591]:
from tqdm import tqdm

def pick_shortest(roots):

    children = {root: [root] for root in roots}
    scores = {root: len(root) for root in roots}
    
    for m in range(3):
        # print("Depth: ", m)
        for root in roots:  
            current_children = children[root]

            # print("current_children: ", current_children)
           
            # compute next children
            next_children = []
            for child in current_children:
                next_children.extend([second_sequence(child)[0]])

            # print("Next children: ", next_children) 
            # filter
            next_children = [child for child in next_children if len(child) == min([len(child) for child in next_children])]
            
            # update children
            children[root] = next_children

            # update scores
            scores[root] = len(next_children[0])

        # Check if there is root with unique minimum 
        min_score = min([scores[root] for root in roots])
        min_roots = [root for root in roots if scores[root] == min_score]
        # print(f"{min_roots}, {scores}")

        if len(min_roots) == 1:
            # print(f"Certificate: {min_roots}, {scores}")
            return min_roots[0]
        
    return min_roots

In [592]:
# Optimal sequences on the dir pad 
precomputed_dir_sequences = {}

for key, sequence in keypad_sequences.items():
    shortest = pick_shortest(sequence)
    precomputed_dir_sequences[key] = shortest

precomputed_dir_sequences

{'A^': '<A',
 'A>': 'vA',
 'Av': '<vA',
 'A<': 'v<<A',
 '^A': '>A',
 '>A': '^A',
 'vA': '^>A',
 '<A': '>>^A',
 'AA': 'A',
 '^>': 'v>A',
 '^v': 'vA',
 '^<': 'v<A',
 '>^': '<^A',
 'v^': '^A',
 '<^': '>^A',
 '^^': 'A',
 'v<': '<A',
 '<v': '>A',
 'v>': '>A',
 '>v': '<A',
 'vv': 'A',
 '<>': '>>A',
 '><': '<<A',
 '<<': 'A',
 '>>': 'A'}

In [611]:
def get_optimal_path(sequence, start='A'):
    if start != sequence[0]:
        sequence = start + sequence
    result = ''
    for i in range(len(sequence)-1):
        seq = precomputed_dir_sequences[sequence[i:i+2]]
        result = result + seq

    return result

get_optimal_path('<A^A>^^AvvvA')

'v<<A>>^A<A>AvA<^AA>A<vAAA^>A'

### Check that I can reproduce Pt 1 with precomputed dirs? 

**Reproducing a test case first**

In [688]:
test_inputs = ["029A",
"980A",
"179A",
"456A",
"379A"]

In [689]:
def get_answer(inputs): 
    answer = dict()  # key is the num input, value is an optimal path

    for input in inputs:
        long_paths = []

        for root in roots(input): # root is a sequence on the num pad
            path = get_optimal_path(root)
            path2 = get_optimal_path(path)

            long_paths.append(path2)

        shortest_long_path = min(long_paths, key=lambda x: len(x))
        answer[input] = shortest_long_path

    return answer

In [759]:
complexity(get_answer(test_inputs))

126384

**Reproducing Pt.1**

In [691]:
codes = []
with open("input.txt") as f: 
    for line in f: 
        codes.append(line.strip())
codes

complexity(get_answer(codes)) # Sick! Finally

157892

### Part 2 solution: 

Ideas: 
* memoization 
* update dict that store sequence lengths instead of sequences?

In [692]:
def get_answer_to_pt2(inputs, depth = 1): 
    answer = dict()  # key is the num input, value is an optimal path

    for input in inputs:
        long_paths = []

        for root in roots(input): # root is a sequence on the num pad
            path = get_optimal_path(root)

            for _ in range(depth):
                path = get_optimal_path(path)
    
            long_paths.append(path)

        # Note: this only goes over roots.
        shortest_long_path = min(long_paths, key=lambda x: len(x))
        answer[input] = shortest_long_path

    return answer

In [693]:
get_optimal_path('A^', start='A')

'<A'

In [ ]:
def complexity(answer):
    result = 0

    for code, path in answer.items():
        result += len(path) * int(code[:-1])

    return result

In [694]:
complexity(get_answer_to_pt2(codes, 5))

5954180

### Memoize optimal sequences?

In [695]:
# memoize sequences of key strokes on the second keypad that control the first keypad

memo = dict() 

for key, sequence in precomputed_dir_sequences.items():
    memo[key] = get_optimal_path(sequence, start='A')

## Does a dict representation work here?

In [723]:
test_seq = '<A^A>^^AvvvA'

def seq_to_dict(seq):
    """
    Convert test seq to a dict of pairs : counts
    """
    if seq[0] != 'A':
        seq = 'A' + seq

    elif seq == 'A':
        return {'AA': 1}

    result = dict()
    for i in range(len(seq)-1):
        result.setdefault(seq[i:i+2], 0)
        result[seq[i:i+2]] += 1

    return result

def iterate_seq_dict(input_dict): 
    """
    Iterate the dictionary of pairs: counts

    Given an input dict that stores the counts for each path on the sequence, 
    generate the next iteration of the dictionary
    """
    new_dict = dict() 

    for key, value in input_dict.items():
       
        path =  precomputed_dir_sequences[key]
        subdict = seq_to_dict(path)
       
        for subkey, subvalue in subdict.items():
            new_dict.setdefault(subkey, 0)
            new_dict[subkey] += subvalue*value

    return new_dict
    

Small test example with <A^^A>

In [724]:
test_dict = seq_to_dict('<A^^A>')   

In [725]:
print(test_dict)

{'A<': 1, '<A': 1, 'A^': 1, '^^': 1, '^A': 1, 'A>': 1}


In [726]:
new_dict = dict() 

path_check = ''
for key, value in test_dict.items():
    print(key)
    path =  precomputed_dir_sequences[key]
    path_check = path_check + path
    subdict = seq_to_dict(path)
    print(path, subdict)
    for subkey, subvalue in subdict.items():
        new_dict.setdefault(subkey, 0)
        new_dict[subkey] += subvalue

new_dict

A<
v<<A {'Av': 1, 'v<': 1, '<<': 1, '<A': 1}
<A
>>^A {'A>': 1, '>>': 1, '>^': 1, '^A': 1}
A^
<A {'A<': 1, '<A': 1}
^^
A {'AA': 1}
^A
>A {'A>': 1, '>A': 1}
A>
vA {'Av': 1, 'vA': 1}


{'Av': 2,
 'v<': 1,
 '<<': 1,
 '<A': 2,
 'A>': 2,
 '>>': 1,
 '>^': 1,
 '^A': 1,
 'A<': 1,
 'AA': 1,
 '>A': 1,
 'vA': 1}

In [727]:
assert(iterate_seq_dict(test_dict) == new_dict == seq_to_dict(path_check)) # Sanity check

## Larger example

First, check that the iteration on dict commutes correctly

In [741]:
test_root = roots(codes[0])[0]
get_optimal_path(test_root)

A = seq_to_dict(get_optimal_path(test_root))
B = iterate_seq_dict(seq_to_dict(test_root))

assert(A == B)
assert(len(get_optimal_path(test_root)) == sum(A.values()))

### Check that I can reproduce Pt.1

#### Test inputs:

In [764]:
test_inputs = ["029A",
"980A",
"179A",
"456A",
"379A"]

def get_answer_from_dicts(inputs, depth = 2): 
    answer = dict()  # key is the num input, value is an optimal path

    for input in inputs:
        long_paths = []

        for root in roots(input): # root is a sequence on the num pad
            path_dict = seq_to_dict(root)

            for _ in range(depth):
                path_dict = iterate_seq_dict(path_dict)

            long_paths.append(sum(path_dict.values()))

        shortest_long_path = min(long_paths)
        answer[input] = shortest_long_path

    return answer

def complexity_form_dicts(answer):
    result = 0 
    for key, value in answer.items():
        result += value * int(key[:-1])

    return result

complexity_form_dicts(get_answer_from_dicts(test_inputs))

126384

#### Inputs 

In [766]:
codes = []
with open("input.txt") as f: 
    for line in f: 
        codes.append(line.strip())
codes

complexity_form_dicts(get_answer_from_dicts(codes)) # Correcto

157892

In [771]:
complexity_form_dicts(get_answer_from_dicts(codes, depth=25)) # FK YEAH

197015606336332